In [3]:
import pandas as pd
import numpy as np
import geopandas as gpd
from datetime import datetime
from geobr import read_municipality

In [ ]:
base = pd.read_csv("C:/Users/ree11390/OneDrive - M DIAS BRANCO S.A. INDUSTRIA E COMERCIO DE ALIMENTOS/Área de Trabalho/TRABALHO FACULDADE/Dengue Angra.csv", sep=",")
gdf = gpd.read_file("https://geoftp.ibge.gov.br/organizacao_do_territorio/malhas_territoriais/malhas_municipais/municipio_2022/Brasil/BR/BR_Municipios_2022.zip")
municipios_rj = {row["CD_MUN"]: row["NM_MUN"] for _, row in gdf.iterrows() if row["SIGLA_UF"] == "RJ"}

semana_atual_menos_um = datetime.today().isocalendar()[1] - 1

bases = []

url = "https://info.dengue.mat.br/api/alertcity"

for i, (cd_mun, nome_cidade) in enumerate(municipios_rj.items(), start=1):
    print(f"Lendo cidade {i}: {nome_cidade} | Código do Município: {cd_mun}")
    search_filter =(f'geocode={cd_mun}&disease=dengue&format=csv&ew_start=1&ew_end={semana_atual_menos_um}&ey_start=2024&ey_end=2025')
    base = pd.read_csv('%s?%s' % (url, search_filter),
                       usecols=[
                           'data_iniSE',
                           'SE',
                           'casos',
                           'nivel',
                           'pop',
                           'tempmin',
                           'tempmed',
                           'tempmax',
                           'umidmin',
                           'umidmed',
                           'umidmax'
                       ])
    base["municipios"] = nome_cidade
    bases.append(base)

base_compilada = pd.concat(bases, ignore_index=True)

In [20]:
base_coordenadas = gdf[gdf['SIGLA_UF'] == 'RJ'].copy()
base_coordenadas = base_coordenadas.to_crs(epsg=4326)
base_coordenadas["LATITUDE"] = base_coordenadas.centroid.y
base_coordenadas["LONGITUDE"] = base_coordenadas.geometry.centroid.x
base_coordenadas = base_coordenadas[['NM_MUN', 'LATITUDE', 'LONGITUDE']]
base_coordenadas.columns = ['municipios', 'latitude', 'longitude']
base_coordenadas = base_coordenadas.reset_index(drop=False)

base_compilada = base_compilada.merge(base_coordenadas, on="municipios", how='left')

display(base_compilada)

C:\Users\ree11390\AppData\Local\Temp\ipykernel_18276\1302854216.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  base_coordenadas["LATITUDE"] = base_coordenadas.centroid.y
C:\Users\ree11390\AppData\Local\Temp\ipykernel_18276\1302854216.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  base_coordenadas["LONGITUDE"] = base_coordenadas.geometry.centroid.x


,data_iniSE,SE,casos,nivel,pop,tempmin,umidmax,umidmed,umidmin,tempmed,tempmax,municipios,index,latitude,longitude
0,2025-03-23,202513,26,4,181228.0,NaN,NaN,NaN,NaN,NaN,NaN,Angra dos Reis,3175,-22.986551,-44.352753
1,2025-03-16,202512,196,4,181228.0,19.794900,95.257500,89.133460,79.001520,21.492720,23.633080,Angra dos Reis,3175,-22.986551,-44.352753
2,2025-03-09,202511,221,4,181228.0,20.940557,94.923943,87.444543,74.365257,23.051143,26.434914,Angra dos Reis,3175,-22.986551,-44.352753
3,2025-03-02,202510,245,4,181228.0,20.426286,93.506557,78.475886,56.525414,23.367243,27.048471,Angra dos Reis,3175,-22.986551,-44.352753
4,2025-02-23,202509,162,4,181228.0,20.991543,93.823657,76.639186,53.078614,24.380486,28.293343,Angra dos Reis,3175,-22.986551,-44.352753
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5975,2024-01-28,202405,1436,4,270543.0,19.314829,97.578986,82.970971,59.098243,23.502643,28.677814,Volta Redonda,3266,-22.490923,-44.088931
5976,2024-01-21,202404,1429,4,270543.0,19.284343,95.400500,85.967843,72.504929,21.759757,24.619329,Volta Redonda,3266,-22.490923,-44.088931
5977,2024-01-14,202403,1085,4,270543.0,21.732000,95.614271,79.886643,60.324100,26.021371,30.661743,Volta Redonda,3266,-22.490923,-44.088931
5978,2024-01-07,202402,538,4,270543.0,21.247557,94.572400,78.776800,58.949829,25.432443,29.893400,Volta Redonda,3266,-22.490923,-44.088931


In [21]:
base_compilada.to_csv('base_compilada.csv', sep=';', index=False)

In [4]:
base_compilada = pd.read_csv('C:/Users/ree11390/OneDrive - M DIAS BRANCO S.A. INDUSTRIA E COMERCIO DE ALIMENTOS/Área de Trabalho/TRABALHO FACULDADE/bases/base_compilada.csv', sep=';')

In [5]:
base_compilada.to_json("C:/Users/ree11390/OneDrive - M DIAS BRANCO S.A. INDUSTRIA E COMERCIO DE ALIMENTOS/Área de Trabalho/TRABALHO FACULDADE/bases/base_compilada_jason.json",
                       orient='records',
                       force_ascii=False)